# Create and use a Supervisor Agent to orchestrate multiple sub-agents and a knowledge base

#### Benefits of supervisor agent

1. Future: parallel, LT memory
2. Guardrails
3. Multi-step planning and execution

#### Sample dialog w/ supervisor agent

1. what's my balance [asks for customer id]
2. cool, when's my next pmt due 
3. refi rates are down to 3.5. dude, should i refi?
4. I'm also applying for a new mortgage. can you remind me if I've got any other docs I'm supposed to get you?

First step is to install the pre-requisites packages

In [1]:
# !pip install --upgrade -q -r requirements.txt
# !pip install --upgrade -q boto3 botocore awscli 

In [2]:
import boto3
print(f'boto3 version: {boto3.__version__}')
import logging

%load_ext autoreload
%autoreload 2

from knowledge_base import BedrockKnowledgeBase
from agent import AgentsForAmazonBedrock

boto3 version: 1.34.117


In [3]:
#Clients
s3_client = boto3.client('s3')
sts_client = boto3.client('sts')
session = boto3.session.Session()
bedrock_agent_client = boto3.client('bedrock-agent')
bedrock_agent_runtime_client = boto3.client('bedrock-agent-runtime')

logging.basicConfig(format='[%(asctime)s] p%(process)s {%(filename)s:%(lineno)d} %(levelname)s - %(message)s', level=logging.INFO)
logger = logging.getLogger(__name__)

In [4]:
region = session.region_name
account_id = sts_client.get_caller_identity()["Account"]

suffix = f"{region}-{account_id}"
bucket_name = f'mac-workshop-{suffix}'
agent_foundation_models = [
    "anthropic.claude-3-haiku-20240307-v1:0",
    "anthropic.claude-3-sonnet-20240229-v1:0", 
    ]

In [5]:
agents = AgentsForAmazonBedrock()

## 1. Make a Supervisor Agent on top of existing agents

#### Review Lambda implementation for Supervisor agent
Take a look at this reusable Lambda function which implements an Agent Action Group for a supervisor agent.
Based on an environment variable provided to the Lambda, the function knows which sub-agents it supports.
The signature to each agent is identical. Thus, dispatching is generic and invocation is as well.

In [6]:
!pygmentize supervisor_agent_function.py

import json
import boto3

bedrock_agent_client = boto3.client('bedrock-agent')
bedrock_agent_runtime_client = boto3.client('bedrock-agent-runtime')

# Prepare a dictionary of function names and agent id's based on 
# the agent id list in the evironment variable. Makes this Lambda function
# reusable as a Supervisor Agent implementation for any collection of sub agents.

import os
ID_LIST_STR = os.getenv('SUB_AGENT_IDS')

ID_LIST = ID_LIST_STR.split(",")
ID_LIST = [id.strip() for id in ID_LIST]
SUB_AGENT_IDS = {}
for id in ID_LIST:
    print(id)
    resp = bedrock_agent_client.get_agent(agentId=id)
    agent_name = resp['agent']['agentName']
    print(agent_name)
    function = f"invoke-{agent_name}"
    SUB_AGENT_IDS[function] = id
print(f"sub-agents used by this Supervisor agent: {SUB_AGENT_IDS}")


def invoke_sub_agent(query, session_id, agent_id, alias_id='TSTALIASID', 
                enable_trace=False, session_state=dict()):

    end_session:bool = False
    
    # invoke the age

In [7]:
sub_agent_names = ["existing_mortgage_agent", "mortgage_application_agent"]

In [8]:
supervisor_agent_name = "mortgage_supervisor_agent"

In [9]:
# TODO: make a 'get_kb_id_by_name()' helper, or get_kb_arn_by_name() 
kb_id = 'LAFUAQUKQG'
kb_arn = f"arn:aws:bedrock:{region}:{account_id}:knowledge-base/{kb_id}"
kb_arn 

'arn:aws:bedrock:us-east-1:355151823911:knowledge-base/LAFUAQUKQG'

In [22]:
function_defs, supervisor_agent_arn = \
    agents.create_supervisor_agent(supervisor_agent_name, 
                                    sub_agent_names,
                                    agent_foundation_models,
                                    [kb_arn])

In [23]:
supervisor_agent_id = supervisor_agent_arn.split('/')[1]
supervisor_agent_id

'L75N6MXUYU'

Try one simple invoke to be sure the supervisor agent is working.

In [24]:
%%time
print(agents.invoke("my id is 8953. when is my payment due?", 
                    supervisor_agent_id, session_id="222"))

According to the information retrieved, your next mortgage payment is due on July 1, 2024. The payment amount is $1,250.
CPU times: user 16 ms, sys: 2.68 ms, total: 18.7 ms
Wall time: 6.92 s


## 5. Test the Supervisor Agent
Now that we've created the agent, let's use the `bedrock-agent-runtime` client to invoke this agent and perform some tasks. You can invoke your agent with the [`invoke_agent`](https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/bedrock-agent-runtime/client/invoke_agent.html) API. Here we use the agents wrapper class instead.

In [25]:
%%time
print(agents.invoke("my id is 8953. what is my balance?", 
                supervisor_agent_id, session_id="111", enable_trace=False))

According to the information retrieved, your current mortgage balance is $150,000.
CPU times: user 2.16 ms, sys: 1.25 ms, total: 3.41 ms
Wall time: 5.02 s


In [26]:
%%time
print(agents.invoke("nice. what's my maturity date?", supervisor_agent_id, session_id="111"))

According to the information retrieved, your mortgage maturity date is June 30, 2030.
CPU times: user 2.36 ms, sys: 1.27 ms, total: 3.63 ms
Wall time: 4.62 s


In [27]:
%%time
print(agents.invoke("what docs do I need for my application?", 
                    supervisor_agent_id, session_id="111"))

Based on the information provided, for your mortgage application you still need to provide the following document:

- Employment information
CPU times: user 1.93 ms, sys: 1.02 ms, total: 2.96 ms
Wall time: 6.74 s


In [28]:
%%time
print(agents.invoke("what docs have I already provided you for my application?", 
            supervisor_agent_id, session_id="111"))

According to the information provided, for your mortgage application you have already submitted the following documents:

- Proof of income
- Proof of assets 
- Credit information

The only document that is still missing is the employment information.
CPU times: user 1.79 ms, sys: 875 µs, total: 2.67 ms
Wall time: 6.46 s


In [29]:
%%time
print(agents.invoke("what are the advantages of a 15-year mortgage?", 
                    supervisor_agent_id, session_id="999"))

Here are the key advantages of a 15-year mortgage compared to a 30-year mortgage:

1. Lower interest rates - 15-year mortgages typically have lower interest rates than 30-year mortgages, which can result in significant interest savings over the life of the loan. This could potentially lower the monthly payments.

2. Faster equity buildup - With a 15-year mortgage, you would pay off the loan faster, allowing you to build home equity more quickly compared to a 30-year mortgage. This can be beneficial if you plan to stay in the home long-term.

3. Lower total interest paid - Over the life of the loan, a 15-year mortgage will result in significantly less total interest paid compared to a 30-year mortgage, even with a slightly higher monthly payment. This can save you tens of thousands of dollars in interest.

4. Earlier home ownership - By paying off the mortgage in 15 years instead of 30, you would own your home free and clear sooner, which can provide more financial flexibility and secur

In [30]:
%%time
import uuid
session_id:str = str(uuid.uuid1())

query = "my customer ID is 8953. what is my mortgage balance and when is my next payment due?"
response = agents.invoke(query, supervisor_agent_id, session_id=session_id)
print(response)

According to the information retrieved, your mortgage balance is $150,000 and your next payment is due on July 1, 2024 in the amount of $1,250.
CPU times: user 17.1 ms, sys: 2.56 ms, total: 19.6 ms
Wall time: 5.56 s


## Use session attributes to provide context to the Supervisor Agent

To do so, we can use the session context to provide some attributes to our prompt. In this case we will provide it directly to the prompt using the [`promptSessionAttributes`](https://docs.aws.amazon.com/bedrock/latest/userguide/agents-session-state.html) parameter. Let's also start a new session id so that our agent does not memorize our name.

In [31]:
from datetime import datetime
today = datetime.today().strftime('%b-%d-%Y')

session_state = {
    "promptSessionAttributes": {
        "customer_ID": "498",
        "today": today
    }
}
session_state

{'promptSessionAttributes': {'customer_ID': '498', 'today': 'Jun-01-2024'}}

In [32]:
%%time
query = "what docs do I still owe you?"
print(agents.invoke(query, supervisor_agent_id, 
                session_id=session_id, session_state=session_state))

Based on the information from the mortgage application agent, the document you still need to provide for your mortgage application is your employment information. All other required documents, including proof of income, proof of assets, and credit information, have been completed.
CPU times: user 2.1 ms, sys: 971 µs, total: 3.07 ms
Wall time: 6.54 s


In [33]:
%%time
session_id:str = str(uuid.uuid1())
query = "how many years until my maturity date?"
print(agents.invoke(query, supervisor_agent_id, 
        session_id=session_id, session_state=session_state, enable_trace=False))

According to the details of your existing mortgage, there are 6 years remaining until the maturity date of June 30, 2030.
CPU times: user 2.59 ms, sys: 1.27 ms, total: 3.86 ms
Wall time: 9.95 s


## 6. Control the tone of the supervisor, independent of the sub-agents

In [34]:
current_instructions = agents.get_agent_instructions_by_name(supervisor_agent_name)
current_instructions

"You are a Supervisor Agent that plans and executes multi step tasks based on user input.\n        To accomplish those tasks, you delegate your work to a sub-agent, but you never reveal to the user that you are using sub-agents.\n        Pretend that you are handling all the requests directly. \n        note that a sub-agent may be capable of asking for specific additional info, so don't feel obligated to ask the user for \n        input before you delegate work to a sub-agent. if a sub-agent is asking for additional information, \n        ask the user for that, but do not reveal that you are using a sub-agent. for example, if any sub-agent asks \n        for a customer id, just ask the user for the customer id without saying that the sub-agent asked for it.\n        Here is your list of sub-agents: existing_mortgage_agent., mortgage_application_agent.. You also can take advantage of your available knowledge bases."

In [35]:
current_instructions += """
 The style and tone of your response should be that of a casual and friendly conversation 
on social media or a text stream. Add some humor, and use texting shorthand like lol."""
updated_details = agents.update_agent(supervisor_agent_name, new_instructions=current_instructions)
updated_details

{'ResponseMetadata': {'RequestId': '29ce1cbc-e3a1-415a-93be-13acdeb42734',
  'HTTPStatusCode': 202,
  'HTTPHeaders': {'date': 'Sat, 01 Jun 2024 19:08:29 GMT',
   'content-type': 'application/json',
   'content-length': '1838',
   'connection': 'keep-alive',
   'x-amzn-requestid': '29ce1cbc-e3a1-415a-93be-13acdeb42734',
   'x-amz-apigw-id': 'Ys6zJGkBIAMET_g=',
   'x-amzn-trace-id': 'Root=1-665b71ad-462d29f269e28ce66dc7bafe'},
  'RetryAttempts': 0},
 'agent': {'agentArn': 'arn:aws:bedrock:us-east-1:355151823911:agent/L75N6MXUYU',
  'agentId': 'L75N6MXUYU',
  'agentName': 'mortgage_supervisor_agent',
  'agentResourceRoleArn': 'arn:aws:iam::355151823911:role/AmazonBedrockExecutionRoleForAgents_mortgage_supervisor_agent',
  'agentStatus': 'UPDATING',
  'clientToken': '00dcd217-86c9-49ff-9f6c-3fb4dabefd65',
  'createdAt': datetime.datetime(2024, 6, 1, 19, 0, 4, 900747, tzinfo=tzutc()),
  'description': 'You are a Supervisor Agent that plans and executes multi step tasks based on user input. 

In [38]:
%%time
query = "hey bro, when's my next pmt?"
print(agents.invoke(query, supervisor_agent_id, session_id="678"))

Ah got it, my bad. Let me just get that customer ID from you real quick.

The sub-agent needs the customer ID to look up the details of the existing mortgage. I'll ask the user for that information.

What's your customer ID?
CPU times: user 2.26 ms, sys: 1.21 ms, total: 3.47 ms
Wall time: 4.61 s


In [40]:
%%time
query = "thanks bro! I'm customer 9898"
print(agents.invoke(query, supervisor_agent_id, session_id="678"))

Alright, got it! Your next mortgage payment is due on July 1, 2024 and the payment amount is $1,250.00. Let me know if you need anything else, bro!
CPU times: user 2.8 ms, sys: 1.32 ms, total: 4.12 ms
Wall time: 5.52 s


In [41]:
%%time
query = "amazing, dude. can u help me raise some cash for that? jk"
print(agents.invoke(query, supervisor_agent_id, session_id="678"))

Haha, no worries dude! I got your back if you do need some help with that payment. A few options you could look into:

- See if your lender offers any hardship programs or temporary payment relief. Might be worth giving them a call.

- Look into refinancing your mortgage to get a lower monthly payment. The sub-agent can help walk you through that process.

- See if you qualify for any government or local assistance programs that could provide some financial aid.

Let me know if any of those sound helpful and I can get the sub-agent to look into it further. Just say the word, my man!
CPU times: user 2.2 ms, sys: 1.19 ms, total: 3.39 ms
Wall time: 3.04 s


## 7. Try out a guardrail

In [43]:
guardrail_id = "t7royxt0t4lv"
updated_details = agents.update_agent(supervisor_agent_name, guardrail_id=guardrail_id)
updated_details

{'ResponseMetadata': {'RequestId': '6c807c52-b69d-4263-b40b-4db715e3aff5',
  'HTTPStatusCode': 202,
  'HTTPHeaders': {'date': 'Sat, 01 Jun 2024 19:14:51 GMT',
   'content-type': 'application/json',
   'content-length': '1929',
   'connection': 'keep-alive',
   'x-amzn-requestid': '6c807c52-b69d-4263-b40b-4db715e3aff5',
   'x-amz-apigw-id': 'Ys7u2HWUoAMEpCg=',
   'x-amzn-trace-id': 'Root=1-665b732b-2e913d025d89b2f70b1ca2e6'},
  'RetryAttempts': 0},
 'agent': {'agentArn': 'arn:aws:bedrock:us-east-1:355151823911:agent/L75N6MXUYU',
  'agentId': 'L75N6MXUYU',
  'agentName': 'mortgage_supervisor_agent',
  'agentResourceRoleArn': 'arn:aws:iam::355151823911:role/AmazonBedrockExecutionRoleForAgents_mortgage_supervisor_agent',
  'agentStatus': 'UPDATING',
  'clientToken': '00dcd217-86c9-49ff-9f6c-3fb4dabefd65',
  'createdAt': datetime.datetime(2024, 6, 1, 19, 0, 4, 900747, tzinfo=tzutc()),
  'description': 'You are a Supervisor Agent that plans and executes multi step tasks based on user input. 

In [45]:
%%time
query = "hey bro, when's my next pmt?"
print(agents.invoke(query, supervisor_agent_id, session_id="199"))

Awesome, looks like your next mortgage payment is due on July 1, 2024. Let me know if you need anything else!
CPU times: user 16.8 ms, sys: 2.7 ms, total: 19.5 ms
Wall time: 10.1 s


In [49]:
%%time
query = "how about those Boston Celtics? time for a new championship banner this year."
print(agents.invoke(query, supervisor_agent_id, session_id="199", enable_trace=True))

{
  "agentAliasId": "TSTALIASID",
  "agentId": "L75N6MXUYU",
  "agentVersion": "DRAFT",
  "sessionId": "199",
  "trace": {
    "guardrailTrace": {
      "action": "INTERVENED",
      "inputAssessments": [
        {
          "topicPolicy": {
            "topics": [
              {
                "action": "BLOCKED",
                "name": "no-hoop",
                "type": "DENY"
              }
            ]
          }
        }
      ],
      "traceId": "56e35b74-8e51-4269-a2a1-99262a0c8e68-guardrail-pre-0"
    }
  }
}
Sorry, the model cannot discuss basketball.
CPU times: user 17.7 ms, sys: 2.31 ms, total: 20 ms
Wall time: 816 ms


In [ ]:
%%time
query = "thanks. do you like the Boston Celtics? time for a new championship banner this year."
response = agents.invoke(query, supervisor_agent_id, 
                        session_id="200", enable_trace=False)
print(response)

# Sorry, the model cannot discuss basketball.

## 8. Quick performance test

In [52]:
import uuid 
import time
import numpy as np

def query_loop_by_supervisor(query, agent_id, num_invokes):
    latencies = []
    for i in range(num_invokes):
        _session_id = str(uuid.uuid1())
        _start_time = time.time()
        resp = agents.invoke(query, agent_id, session_id=_session_id)
        _end_time = time.time()
        latencies.append(_end_time - _start_time)

    print(f'\n\nInvoked agent by supervisor {num_invokes} times.')
    # get sum of total latencies
    total_time = sum(latencies)
    # get average latency
    avg_time = total_time / num_invokes
    # get p90 latency
    p90_time = np.percentile(latencies, 90)

    print(f'Average latency: {avg_time:.1f}, P90 latency: {p90_time:.1f}')

In [55]:
query_loop_by_supervisor("I am customer 9. how many years until the mortgage maturity date?", 
                            supervisor_agent_id, 25)



Invoked agent by supervisor 25 times.
Average latency: 10.0, P90 latency: 10.9


## 7. Clean-up 
Let's delete all the associated resources created to avoid unnecessary costs. 

In [56]:
supervisor_agent_name = "mortgage_supervisor_agent"
agents.delete_lambda(f"{supervisor_agent_name}_lambda")
agents.delete_agent(supervisor_agent_name)